In [1]:
import mysql.connector
from sqlalchemy import create_engine
from config import MYSQL_CONNECTION, SQLALCH1, SQLALCH2

In [2]:
db = mysql.connector.connect(**MYSQL_CONNECTION)
cur = db.cursor()

con1 = create_engine(SQLALCH1)

con = create_engine(SQLALCH2)

In [3]:
import pandas as pd
from transliterate import translit
from tqdm.auto import tqdm

## copy

In [4]:
tablenames = [
#     "author_name",
    "authors",
    "author_wd_features",
    "author_wiki_data",
    "author_wiki_properties_labels",
    "geo_production",
    "geo_wiki",
    "images",
    "museums",
    "technologies",
    "technology_name",
    "__tech_raw",
    "typology"
]

In [5]:
df = pd.read_sql_table("author_name", con=SQLALCH1)

In [6]:
def to_en(x):
    if x is None:
        return None
    return translit(x, language_code="ru", reversed=True)

def to_ru(x):
    if x is None:
        return None
    return translit(x, language_code="ru")

In [7]:
df["name_en"] = df[["name_en", "name_ru"]].apply(
    lambda x: to_en(x["name_ru"]) if x["name_en"] is None else x["name_en"], axis=1
)

In [8]:
df["name_ru"] = df[["name_en", "name_ru"]].apply(
    lambda x: to_en(x["name_en"]) if x["name_ru"] is None else x["name_ru"], axis=1
)

In [9]:
to_ru("Luigi Rossini")

'Луиги Россини'

In [10]:
df[df["name_ru"].isnull()]

,id,name_en,name_ru,wiki_id
29,30,None,None,Q81644873
39,40,None,None,Q20604405
262,263,None,None,Q16432712
310,311,None,None,Q13209494
446,447,None,None,Q20617698
...,...,...,...,...
56220,56221,None,None,Q12131926
56252,56253,None,None,Q12562083
56298,56299,None,None,Q12150387
56300,56301,None,None,Q12087800


In [11]:
def get_en_label(x):
    entity = client.get(x, load=True)
#     print(x)
    return entity.label.get("en")

In [12]:
from wikidata.client import Client
client = Client()

In [13]:
tqdm.pandas(desc="")

/home/dkbrz/.local/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
df["name_en"] = df[["name_en", "wiki_id"]].progress_apply(
    lambda x: get_en_label(x["wiki_id"]) if x["name_en"] is None else x["name_en"], axis=1
)

HTTPError: HTTP Error 404: Not Found

In [ ]:
df["name_ru"] = df[["name_en", "name_ru"]].apply(
    lambda x: to_en(x["name_en"]) if x["name_ru"] is None else x["name_ru"], axis=1
)

In [ ]:
# for x in df[df["name_ru"].isnull()]["name_en"]:
#     client.get(x, load=True).label

m'Harald Böttger'

In [28]:
df.shape

(56333, 4)